In [25]:
import os
import json
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from datetime import datetime



In [26]:
# Load .env environment variables
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [27]:
def initialize_llm():
    provider = os.getenv("MODEL_PROVIDER")
    if provider == "GROQ":
        print(1)
        # Initialize the Groq LLM
        llm = ChatGroq(
            model= os.getenv("GROQ_MODEL"),
            api_key=os.getenv("GROQ_API_KEY"),
            temperature=0.1,
            max_tokens=5000,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
        )
    elif provider == "OPENAI":
        # Initialize the OpenAI LLM
        llm = ChatOpenAI(
            model= os.getenv("OPENAI_MODEL"),
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            temperature=0.1,
            max_tokens=5000,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
        )
    return llm

In [28]:
llm = initialize_llm()

In [29]:
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x00000176901DAFC0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000176902738C0> root_client=<openai.OpenAI object at 0x000001768FC7F860> root_async_client=<openai.AsyncOpenAI object at 0x000001769032CF20> model_name='gpt-4.1-mini' temperature=0.1 model_kwargs={} openai_api_key=SecretStr('**********') presence_penalty=0.0 frequency_penalty=0.0 top_p=0.95 max_tokens=5000


In [30]:
Executive_Summary_PROMPT = """ 
Student Data: 
Name: {student_name} 
Persona Name: {persona_name} 
Persona strengths: {persona_strengths} 
Persona challenges: {persona_challenges} 
 
SAT Performance: 
Total Score: {current_sat_score_total}  
Accuracy: {current_sat_accuracy} 
Time Management: {current_sat_time_management} 
 
Top 3 Strengths in Math with accuracy: 
{math_top_3} 
 
Top 3 Weaknesses in Math with accuracy: 
{math_bottom_3} 
 
Top 3 Strengths in Reading & Writing with accuracy: 
{rw_top_3} 
 
Top 3 Weaknesses in Reading & Writing with accuracy: 
{rw_bottom_3} 
 
Target SAT Dates: {planned_sat_date} 
Realistic Target Score: {goal_score} 
 
Instructions: 
You are an expert SAT coach and behavioral science specialist. Write a personalized comprehensive SAT readiness report for a Sherpal student using the structured input provided.
Your tone should be **authentic, calm, and professional** — never salesy or exaggerated and matching to **students persona** But not frequently mentioning **students persona** name. Speak to a smart, serious student. The report includes multiple sections and should remain motivational yet concise throughout. Avoid lengthy explanations—prioritize clear, high-impact takeaways over broad commentary - keep it inspiring, focused, and straight to the point. Maintain consistent tone and phrasing across sections to support reliable and repeatable output.
 

Requirements for each section:

**CRITICAL: Return ONLY valid JSON in exactly this structure:** 
Do not wrap the JSON response inside markdown code block tags like ```json or ``` — return only raw JSON :

{{

    "Executive Summary": "Write a brief summary introduction with the student's name and persona name in one short sentence by summarizing their strengths and challenges into 1-2 key words each (e.g., "{persona_name} is a [strength keyword] yet [challenge keyword] learner"). Mention their current SAT score, accuracy, and time management (150 words max).",

    "Key Strengths": 
    {{
        "Reading & Writing: these are the top 3 Reading and writing skills: {rw_top_3}
        output must follow the below structure as array of strings:
        ["<skill1> (<accuracy>%)", "<skill2> (<accuracy>%)", "<skill3> (<accuracy>%)"],

        "Math: these are the top 3 Math skills: {math_top_3}
        output must follow the below structure as array of strings:
        ["<skill1> (<accuracy>%)", "<skill2> (<accuracy>%)", "<skill3> (<accuracy>%)"]

    }},

    "Focus Areas for Growth": {{
        "Reading & Writing: these are the bottom three Reading and Writing skills:{rw_bottom_3}
        output must follow the below structure as array of strings:"
        "["<skill1> (<accuracy>%)", "<skill2> (<accuracy>%)", "<skill3> (<accuracy>%)"]",
        "Math: these are the bottom three Math skills:{math_bottom_3} 
        output must follow the below structure as array of strings:"
        "["<skill1> (<accuracy>%)", "<skill2> (<accuracy>%)", "<skill3> (<accuracy>%)"]"

    }},

    "Bottom 3 Skills Resources:": {{
        "Reading & Writing": these are the bottom 3 skills with resources:{rw_bottom_three_resources}
        output must follow the below structure as array of strings:" 
        [
            "• <skill name> - <link>",
            "• <skill name> - <link>",
            "• <skill name> - <link>"
        ],
        "Math": {math_bottom_three_resources}
        [
            "• <skill name> - <link>",
            "• <skill name> - <link>",
            "• <skill name> - <link>"
        ]
    }}
  }}
"""

In [31]:
FOUNDTION_PHASE_PROMPT = """
You are an expert SAT coach and behavioral science specialist. Write a personalized SAT Foundation Phase study plan, using the structured input provided.
Your tone should be **authentic, calm, and professional** — never salesy or exaggerated and matching to **students persona** But not frequently mentioning **students persona** name. Speak to a smart, serious student. The report includes multiple sections and should remain motivational yet concise throughout.

Create a personalized SAT Foundation Phase study plan for a student named {student_name}. This student is described as a "{persona_name}", with the following strengths: {persona_strengths}, faces these challenges: {persona_challenges}, and growth strategies: {growth_strategies}.

Here is the student's profile and data:
- Name: {student_name}
- Persona Name: {persona_name}
- Key Strengths: {persona_strengths}
- Areas of Improvement: {persona_challenges}
- Growth Strategies: {growth_strategies}

- Current SAT Score: {current_sat_score_total}
- Goal SAT Score: {goal_score}

- Reading & Writing Skill Accuracies (in priority order): {foundation_skills_rw}
- Math Skill Accuracies (in priority order): {foundation_skills_math}


Avoid lengthy explanations—prioritize clear, high-impact takeaways over broad commentary - keep it inspiring, focused, and straight to the point. Maintain consistent tone and phrasing across sections to support reliable and repeatable output.

The Foundation Phase will take place from {foundation_date_range}.

Instructions:
- Do not include '\\n'
- Data should be purely generated based on the Persona details.
- avoid bullet points, lists, or line breaks between sentences.

Purpose:
To build a strong baseline of understanding across all SAT-tested skills with focus on accuracy, concept clarity, and learning discipline.

Key Focus:
- Establish structured study habits
- Improve accuracy and comprehension of core skills
- Identify and start closing skill gaps
- Build confidence slowly through consistency

Using the above data, generate a **Foundation Phase** study report divided into the following sections with these exact headings:

1. Weekly Goals  
2. Practice Methods  
3. Weekly Structure  
4. Flex Day Activities

Requirements for each section:

**CRITICAL: Return ONLY valid JSON in exactly this structure:**
Do not wrap the JSON response inside markdown code block tags like ```json or ``` — return only raw JSON :

{{
    title: "Foundation Phase",
    description:
    [
      {{
         "title": "Weekly Goals",
         "description": "Summarize clear, specific, outcome-driven goals the student should achieve each week during the Foundation Phase. Avoid mentioning how to achieve them or practice methods. Focus on measurable improvements in both Math and Reading & Writing, steady reduction of repeat mistakes, and improved reasoning and accuracy."
      }},
      {{
         "title": "Skill Areas",
         "description":
         [
            {{
               "title": "Math", 
               "description": "List these math skills: {foundation_skills_math}"
            }},
            {{
               "title": "Reading & Writing",
               "description": "List these reading and writing skills: {foundation_skills_rw}"
            }},
         ]
      }},
      {{
         "title": "Practice Methods",
         "description": "For each subject (Reading & Writing and Math), provide detailed, skill-specific practice methods. Use bullet points (starting with '-') for each skill. Mention the skill name in bold or caps, then describe exactly how the student should practice that skill—aligned with their strengths, challenges, and growth strategies. Ensure routines are structured (daily/weekly) and match the learning style of a disciplined student who thrives on consistency while developing adaptability.

         Guidelines:
         - The practice methods should align with these growth strategies: {growth_strategies}
         - The practice method of each skill should align with these skill - summary: {skill_summary_foundation}
         - One sentence from the level method should be included for each skill from this: {skill_level_message_foundation}

         output should be in this format:
         {{
            "title": "Math",
            "description": [
               "<skill>": "practice method for the skill with level_message",
               "<skill>": "practice method for the skill with level_message",
            ],
            "title": "Reading and Writing",
            "description": [
               "<skill>": "practice method for the skill with level_message",
               "<skill>": "practice method for the skill with level_message",
            ],
         }}"
      }},
      {{
         "title": "Weekly Structure",
         "description": "Provide a generic overview of the weekly routine. Explain how each weekday will include focused dual-skill practice (one Math and one Reading & Writing skill), starting with the weakest skills early in the week, midweek focus on moderate skills, Fridays for review, Saturdays for timed full-length practice with review, and one Flex Day for rest and adaptability."
      }},
      {{
         "title": "Flex Day Activities", 
         "description": "Suggest actionable, persona-aligned activities that support rest, reflection, cognitive flexibility, and low-pressure practice. Include reflection on errors, experimenting with new tools or techniques, mixed-skill challenges, physical or mindfulness rest activities, and goal-setting for the next week."
      }}
    ]
}}

**Requirements:**
- Return valid JSON only - no additional text
- Keep descriptions concise but personalized 
- Do not include any generic or vague advice. Use the data provided to personalize every recommendation.
- Make Flex Day Activities unique to the student's persona
"""

In [32]:
ELEVATION_PHASE_PROMPT = """
You are an expert SAT coach and behavioral science specialist. Write a personalized SAT Elevation Phase study plan, using the structured input provided.
Your tone should be **authentic, calm, and professional** — never salesy or exaggerated and matching to **students persona** But not frequently mentioning **students persona** name. Speak to a smart, serious student. The report includes multiple sections and should remain motivational yet concise throughout.

Create a personalized SAT Elevation Phase study plan for a student named {student_name}. This student is described as a "{persona_name}", with the following strengths: {persona_strengths}, faces these challenges: {persona_challenges}, and growth strategies: {growth_strategies}.

Here is the student's profile and data:
- Name: {student_name}
- Persona Name: {persona_name}
- Key Strengths: {persona_strengths}
- Areas of Improvement: {persona_challenges}
- Growth Strategies: {growth_strategies}

- Current SAT Score: {current_sat_score_total}
- Goal SAT Score: {goal_score}

- Reading & Writing Skill Accuracies (in priority order): {elevation_skills_rw}
- Math Skill Accuracies (in priority order): {elevation_skills_math}


Avoid lengthy explanations—prioritize clear, high-impact takeaways over broad commentary - keep it inspiring, focused, and straight to the point. Maintain consistent tone and phrasing across sections to support reliable and repeatable output.

The Elevation Phase will take place from {elevation_date_range}.

Instructions:
- Do not include '\\n'
- Data should be purely generated based on the Persona details.
- avoid bullet points, lists, or line breaks between sentences.

Purpose:
To push skill application under timed conditions, refine accuracy, and improve cognitive flexibility with mid-level and advanced skills.

Key Focus:
- Reduce careless mistakes
- Reinforce second-tier skills (those partially mastered)
- Introduce moderate time pressure
- Layer more strategic practice
- Use performance review cycles (analyze → adapt → retest)

Using the above data, generate a **Elevation Phase** study report divided into the following sections with these exact headings:

1. Weekly Goals  
2. Practice Methods  
3. Weekly Structure  
4. Flex Day Activities

Requirements for each section:

**CRITICAL: Return ONLY valid JSON in exactly this structure:**
Do not wrap the JSON response inside markdown code block tags like ```json or ``` — return only raw JSON :

{{
    title: "Elevation Phase",
    description:
    [
      {{
         "title": "Weekly Goals",
         "description": "Summarize clear, specific, outcome-driven goals the student should achieve each week during the Elevation Phase. Avoid mentioning how to achieve them or practice methods. Focus on measurable improvements in both Math and Reading & Writing, steady reduction of repeat mistakes, and improved reasoning and accuracy."
      }},
      {{
         "title": "Skill Areas",
         "description":
         [
            {{
               "title": "Math", 
               "description": "List these math skills: {elevation_skills_math}"
            }},
            {{
               "title": "Reading & Writing",
               "description": "List these reading and writing skills: {elevation_skills_rw}"
            }},
         ]
      }},
      {{
         "title": "Practice Methods",
         "description": "For each subject (Reading & Writing and Math), provide detailed, skill-specific practice methods. Use bullet points (starting with '-') for each skill. Mention the skill name in bold or caps, then describe exactly how the student should practice that skill—aligned with their strengths, challenges, and growth strategies. Ensure routines are structured (daily/weekly) and match the learning style of a disciplined student who thrives on consistency while developing adaptability. The practice methods should align with these growth strategies: {growth_strategies}:
         
         Guidelines:
         - The practice methods should align with these growth strategies: {growth_strategies}
         - The practice method of each skill should align with these skill - summary: {skill_summary_foundation}
         - One sentence from the level method should be included for each skill from this: {skill_level_message_foundation}

         output should be in this format:
         {{
            "title": "Math",
            "description": [
               "<skill>": "practice method for the skill with level_message",
               "<skill>": "practice method for the skill with level_message",
            ],
            "title": "Reading and Writing",
            "description": [
               "<skill>": "practice method for the skill with level_message",
               "<skill>": "practice method for the skill with level_message",
            ],
         }}"
      }},
      {{
         "title": "Weekly Structure",
         "description": "Provide a generic overview of the weekly routine. Explain how each weekday will include focused dual-skill practice (one Math and one Reading & Writing skill), starting with the weakest skills early in the week, midweek focus on moderate skills, Fridays for review, Saturdays for timed full-length practice with review, and one Flex Day for rest and adaptability."
      }},
      {{
         "title": "Flex Day Activities", 
         "description": "Suggest actionable, persona-aligned activities that support rest, reflection, cognitive flexibility, and low-pressure practice. Include reflection on errors, experimenting with new tools or techniques, mixed-skill challenges, physical or mindfulness rest activities, and goal-setting for the next week."
      }}
    ]
}}

**Requirements:**
- Return valid JSON only - no additional text
- Keep descriptions concise but personalized 
- Do not include any generic or vague advice. Use the data provided to personalize every recommendation.
- Make Flex Day Activities unique to the student's persona
"""

In [ ]:
PEAK_PHASE_PROMPT = """
You are an expert SAT coach and behavioral science specialist. Write a personalized SAT Peak Phase study plan, using the structured input provided.
Your tone should be **authentic, calm, and professional** — never salesy or exaggerated and matching to **students persona** But not frequently mentioning **students persona** name. Speak to a smart, serious student. The report includes multiple sections and should remain motivational yet concise throughout.

Create a personalized SAT Peak Phase study plan for a student named {student_name}. This student is described as a "{persona_name}", with the following strengths: {persona_strengths}, faces these challenges: {persona_challenges}, and growth strategies: {growth_strategies}.

Here is the student's profile and data:
- Name: {student_name}
- Persona Name: {persona_name}
- Key Strengths: {persona_strengths}
- Areas of Improvement: {persona_challenges}
- Growth Strategies: {growth_strategies}

- Current SAT Score: {current_sat_score_total}
- Goal SAT Score: {goal_score}

- Reading & Writing Skill Accuracies (in priority order): {peak_skills_rw}
- Math Skill Accuracies (in priority order): {peak_skills_math}


Avoid lengthy explanations—prioritize clear, high-impact takeaways over broad commentary - keep it inspiring, focused, and straight to the point. Maintain consistent tone and phrasing across sections to support reliable and repeatable output.

The Peak Phase will take place from {peak_date_range}.

Instructions:
- Do not include '\\n'
- Data should be purely generated based on the Persona details.
- avoid bullet points, lists, or line breaks between sentences.

Purpose:
To simulate real test conditions, stabilize performance, and reinforce confidence. Focusing on mastery.

Key Focus:
- Confidence under pressure
- Endurance for full test conditions
- Mental and emotional control
- Pattern recognition
- Minimize score variance
- Lock in strategy

Using the above data, generate a **Peak Phase** study report divided into the following sections with these exact headings:

1. Weekly Goals  
2. Practice Methods  
3. Weekly Structure  
4. Flex Day Activities

Requirements for each section:

**CRITICAL: Return ONLY valid JSON in exactly this structure:**
Do not wrap the JSON response inside markdown code block tags like ```json or ``` — return only raw JSON :

{{
    title: "Peak Phase",
    description:
    [
      {{
         "title": "Weekly Goals",
         "description": "Summarize clear, specific, outcome-driven goals the student should achieve each week during the Peak Phase. Avoid mentioning how to achieve them or practice methods. Focus on measurable improvements in both Math and Reading & Writing, steady reduction of repeat mistakes, and improved reasoning and accuracy."
      }},
      {{
         "title": "Skill Areas",
         "description":
         [
            {{
               "title": "Math", 
               "description": "List these math skills: {peak_skills_math}"
            }},
            {{
               "title": "Reading & Writing",
               "description": "List these reading and writing skills: {peak_skills_rw}"
            }},
         ]
      }},
      {{
         "title": "Practice Methods",
         "description": "For each subject (Reading & Writing and Math), provide detailed, skill-specific practice methods. Use bullet points (starting with '-') for each skill. Mention the skill name in bold or caps, then describe exactly how the student should practice that skill—aligned with their strengths, challenges, and growth strategies. Ensure routines are structured (daily/weekly) and match the learning style of a disciplined student who thrives on consistency while developing adaptability. The practice methods should align with these growth strategies: {growth_strategies}:
         
         Guidelines:
         - The practice methods should align with these growth strategies: {growth_strategies}
         - The practice method of each skill should align with these skill - summary: {skill_summary_foundation}
         - Generate one sentence for each skill describing the student level and his target from this which includes skill level the student is in and what kind of questions they should concentrate on: {skill_level_message_foundation}

         output should be in this format:
         {{
            "title": "Math",
            "description": [
               "<skill>": "practice method for the skill with level_message",
               "<skill>": "practice method for the skill with level_message",
            ],
            "title": "Reading and Writing",
            "description": [
               "<skill>": "practice method for the skill with level_message",
               "<skill>": "practice method for the skill with level_message",
            ],
         }}"
      }},
      {{
         "title": "Weekly Structure",
         "description": "Provide a generic overview of the weekly routine. Explain how each weekday will include focused dual-skill practice (one Math and one Reading & Writing skill), starting with the weakest skills early in the week, midweek focus on moderate skills, Fridays for review, Saturdays for timed full-length practice with review, and one Flex Day for rest and adaptability."
      }},
      {{
         "title": "Flex Day Activities", 
         "description": "Suggest actionable, persona-aligned activities that support rest, reflection, cognitive flexibility, and low-pressure practice. Include reflection on errors, experimenting with new tools or techniques, mixed-skill challenges, physical or mindfulness rest activities, and goal-setting for the next week."
      }}
    ]
}}

**Requirements:**
- Return valid JSON only - no additional text
- Keep descriptions concise but personalized 
- Do not include any generic or vague advice. Use the data provided to personalize every recommendation.
- Make Flex Day Activities unique to the student's persona
"""

In [34]:
TWO_SECTIONS = """
You are an expert SAT coach and behavioral science specialist. Write personalized tips and words of encouragement for SAT Phase study plan, using the structured input provided.
Your tone should be **authentic, calm, and professional** — never salesy or exaggerated and matching to **students persona** But not frequently mentioning **students persona** name. Speak to a smart, serious student. The report includes multiple sections and should remain motivational yet concise throughout.

Create personalized tips and words of encouragement for SAT Phase study plan for a student named {student_name}. This student is described as a "{persona_name}", with the following strengths: {persona_strengths}, faces these challenges: {persona_challenges}, and growth strategies: {growth_strategies}.

Here is the student's profile and data:
- Name: {student_name}
- Persona Name: {persona_name}
- Key Strengths: {persona_strengths}
- Areas of Improvement: {persona_challenges}
- Growth Strategies: {growth_strategies}

- Current SAT Score: {current_sat_score_total}
- Goal SAT Score: {goal_score}

- Reading & Writing Skill Accuracies (in priority order): {foundation_skills_rw}
- Math Skill Accuracies (in priority order): {foundation_skills_math}


Avoid lengthy explanations—prioritize clear, high-impact takeaways over broad commentary - keep it inspiring, focused, and straight to the point. Maintain consistent tone and phrasing across sections to support reliable and repeatable output.
Instructions:
- Do not include '\\n'
- Data should be purely generated based on the Persona details.
- avoid bullet points, lists, or line breaks between sentences.

Requirements for each section:

**CRITICAL: Return ONLY valid JSON in exactly this structure:**
Do not wrap the JSON response inside markdown code block tags like ```json or ``` — return only raw JSON :
{{
    {{
        "title": "Tips to Make It Work Best",
        "description": "Offer 5 high-impact, tactical tips designed to improve the student's adaptability to fast-paced, high-stakes test conditions. Each tip must be grounded in specific behavioral strategies, directly aligned with the student’s persona, strengths, challenges, and growth plan. Focus on skills such as real-time decision-making, learning agility, test-day competitiveness, and strategic adaptability. Examples may include structured error reflection, alternating speed vs. accuracy drills, post-practice confidence logging, or peer-based time-boxed simulations. Tips must be concise, practical, and tailored to the student's learning profile. Avoid vague recommendations or flowery language — every tip should be actionable and clearly measurable. Use this context for full personalization: Persona: {persona_name}, Strengths: {persona_strengths}, Challenges: {persona_challenges}, Growth Strategies: {growth_strategies}, Current Score: {current_sat_score_total}, Goal Score: {goal_score}, Phase Timelines: Foundation: {foundation_date_range}, Elevation: {elevation_date_range}, Peak: {peak_date_range}."
        in this format: 
        [
            "Tip 1 text here.",
            "Tip 2 text here.",
            "Tip 3 text here.",
            "Tip 4 text here.",
            "Tip 5 text here."
        ]
    }},
    {{
        "title": "Words of Encouragement",
        "description": "End with 100–150 words of personalized motivation that speaks to the student’s persona, acknowledging their effort and growth potential. Use empowering, positive language."
    }}
}}

**Requirements:**
- Return valid JSON only - no additional text
- Keep descriptions concise but personalized 
- Do not include any generic or vague advice. Use the data provided to personalize every recommendation.
"""

In [35]:
def list_to_numbered_string(items):
    """Convert list of strings to a numbered string with newlines."""
    return "\n".join([f"{i + 1}. {item}." for i, item in enumerate(items)])

In [36]:
from datetime import datetime
from datetime import datetime, timedelta
 
 
def format_range(start, end):
    return f"{start.strftime('%Y-%m-%d')} - {end.strftime('%Y-%m-%d')}"
 
def get_three_month_study_phases(start_date_str: str):
    try:
        start_date = datetime.strptime(start_date_str, "%Y-%m-%d").date()
    except ValueError:
        return "Invalid date format. Use YYYY-MM-DD."
 
    foundation_start = start_date
    foundation_end = foundation_start + timedelta(days=30)
 
    elevation_start = foundation_end + timedelta(days=1)
    elevation_end = elevation_start + timedelta(days=29)
 
    peak_start = elevation_end + timedelta(days=1)
    peak_end = peak_start + timedelta(days=29)
 
    return (
        format_range(foundation_start, foundation_end),
        format_range(elevation_start, elevation_end),
        format_range(peak_start, peak_end),
        peak_end.strftime('%Y-%m-%d')
    )
 
def get_custom_study_phases(today_str: str, planned_str: str):
    try:
        today = datetime.strptime(today_str, "%Y-%m-%d").date()
        planned = datetime.strptime(planned_str, "%Y-%m-%d").date()
    except ValueError:
        return "Invalid date format. Use YYYY-MM-DD."
 
    if planned <= today:
        return "Planned date must be after today's date."
 
    diff_days = (planned - today).days
 
    part = diff_days // 3
    remainder = diff_days % 3
 
    foundation_start = today
    foundation_end = foundation_start + timedelta(days=part + (1 if remainder > 0 else 0) - 1)
 
    elevation_start = foundation_end + timedelta(days=1)
    elevation_end = elevation_start + timedelta(days=part + (1 if remainder > 1 else 0) - 1)
 
    peak_start = elevation_end + timedelta(days=1)
    peak_end = planned
 
    return (
        format_range(foundation_start, foundation_end),
        format_range(elevation_start, elevation_end),
        format_range(peak_start, peak_end),
        peak_end.strftime('%Y-%m-%d')
    )
 
def get_study_plan(current_score: int, goal_score: int, today_date_str: str, planned_date_str: str) -> str:
    # Convert string to date
    try:
        today_date = datetime.strptime(today_date_str, "%Y-%m-%d").date()
        planned_date = datetime.strptime(planned_date_str, "%Y-%m-%d").date()
    except ValueError:
        return "Invalid date format. Use YYYY-MM-DD."
 
    score_diff = goal_score - current_score
    if score_diff < 0:
        return "Invalid input: Goal score must be higher than current score."
 
    days_gap = (planned_date - today_date).days
    if days_gap < 0:
        return "Invalid input: Planned date must be in the future."
   
    tone_confident_encouraging = "Use a confident and encouraging tone. Focus on fine-tuning and light improvements."
    tone_motivational_urgent = "Use a motivational tone with urgency. Emphasize structured habits and sustained effort."
    tone_serious_constructive = "Use serious, honest, and constructive tone. Highlight foundational review and disciplined rebuilding."
 
    duration_confident = "60"
    flex_duration_confident = "30"
 
    duration_ambitious = "90"
    flex_duration_ambitious = "60"
 
    duration_aggressive = "120"
    flex_duration_aggressive = "90"
 
    fifth_sentence = ""
 
    # Additionally, include these exact points at the end of the summary:
    conclusion = "Conclude the summary by adding these exact sentences: Considering your upcoming SAT exam date and target score, please write an email to info@sherpalai.com so we can assist you in tailoring your study plan. The default study plan is for three months of total preparation time."
 
    # Case 1: Any score difference and gap < 9 days
    if days_gap < 8:
 
        result = get_three_month_study_phases(today_date_str)
        if isinstance(result, str):
            return result
        else:
            foundation, elevation, peak, end_date = result
 
        duration = duration_confident
        flex_duration = flex_duration_confident
        tone = tone_confident_encouraging
        fifth_sentence = conclusion
       
        return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
 
    # Case 2: 9–31 days
    if 8 <= days_gap <= 31:
        if score_diff <= 100:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_confident
            flex_duration = flex_duration_confident
            tone = tone_confident_encouraging
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
       
        elif 101 <= score_diff <= 250:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_ambitious
            flex_duration= flex_duration_ambitious
            tone = tone_motivational_urgent
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
       
        elif 251 <= score_diff <= 400:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
           
            duration= duration_aggressive
            flex_duration= flex_duration_aggressive
            tone = tone_serious_constructive
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
       
        elif score_diff > 400:
 
            result = get_three_month_study_phases(today_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
           
            duration = duration_confident
            flex_duration = flex_duration_confident
            tone = tone_confident_encouraging
            fifth_sentence = conclusion
           
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
 
    # Case 3: 31–60 days
    if 31 < days_gap <= 60:
        if score_diff <= 500:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_confident
            flex_duration = flex_duration_confident
            tone = tone_confident_encouraging
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
        elif 501 <= score_diff <= 600:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_ambitious
            flex_duration= flex_duration_ambitious
            tone = tone_motivational_urgent
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
       
        elif 601 <= score_diff <= 700:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration= duration_aggressive
            flex_duration= flex_duration_aggressive
            tone = tone_serious_constructive
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
        elif score_diff > 700:
 
            result = get_three_month_study_phases(today_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_confident
            flex_duration = flex_duration_confident
            tone = tone_confident_encouraging
            fifth_sentence = conclusion
           
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
 
    # Case 4: 61–90 days
    if 61 <= days_gap <= 90:
        if score_diff <= 800:
           
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_confident
            flex_duration = flex_duration_confident
            tone = tone_confident_encouraging
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
       
        elif 801 <= score_diff <= 900:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_ambitious
            flex_duration= flex_duration_ambitious
            tone = tone_motivational_urgent
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
       
        elif 901 <= score_diff <= 1000:
 
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration= duration_aggressive
            flex_duration= flex_duration_aggressive
            tone = tone_serious_constructive
 
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
        elif score_diff > 1000:
 
            result = get_three_month_study_phases(today_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_confident
            flex_duration = flex_duration_confident
            tone = tone_confident_encouraging
            fifth_sentence = conclusion
           
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
 
    # Case 5: >90 days
    if days_gap > 90:
           
            result = get_custom_study_phases(today_date_str, planned_date_str)
            if isinstance(result, str):
                return result
            else:
                foundation, elevation, peak, end_date = result
 
            duration = duration_confident
            flex_duration = flex_duration_confident
            tone = tone_confident_encouraging
           
            return foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence
 
    return "Unexpected case. Please review the inputs."
 
def get_domain_rankings(rw_domains, math_domains):
    # Weightage for each domain
    rw_weightage = {
        "Craft and Structure": 0.28,
        "Information and Ideas": 0.26,
        "Standard English Conventions": 0.26,
        "Expression of Ideas": 0.20
    }
 
    math_weightage = {
        "Algebra": 0.35,
        "Advanced Math": 0.35,
        "Problem-Solving and Data Analysis": 0.15,
        "Geometry and Trigonometry": 0.15
    }
 
    # Helper to compute domain info with priority score
    def process_domains(domains, weight_map):
        domain_info = []
        for d in domains:
            name = d["domain"]
            accuracy = d["accuracy"]
            weight = weight_map.get(name, 0)
            priority_score = (1 - accuracy) * weight
            domain_info.append({
                "name": name,
                "accuracy": accuracy,
                "weightage": weight,
                "priority_score": priority_score
            })
        return domain_info
 
    # Process each category
    rw_processed = process_domains(rw_domains, rw_weightage)
    math_processed = process_domains(math_domains, math_weightage)
 
    # Combine and sort all domains
    combined_domains = rw_processed + math_processed
 
    combined_ranking = sorted(
        combined_domains,
        key=lambda d: (-d['priority_score'], -d['weightage'], d['name'])
    )
 
    rw_ranking = sorted(
        rw_processed,
        key=lambda d: (-d['priority_score'], -d['weightage'], d['name'])
    )
 
    math_ranking = sorted(
        math_processed,
        key=lambda d: (-d['priority_score'], -d['weightage'], d['name'])
    )
 
    return combined_ranking, rw_ranking, math_ranking

SyntaxError: invalid syntax (1857733344.py, line 189)

In [ ]:
def get_top_and_bottom_skills(skills_data, skill_accuracy_data):
    # Sort by priority (ascending)
    sorted_skills = sorted(skills_data, key=lambda x: int(x["priority"]))
 
    # Helper: Find accuracy from skill_accuracy_data by name
    def get_accuracy(skill_name):
        for item in skill_accuracy_data:
            if item.get("name") == skill_name:
                return item.get("accuracy", "N/A")
        return "N/A"
 
    # Format skill name + accuracy
    def format_with_accuracy(skill_name):
        accuracy = get_accuracy(skill_name)
        return f"{skill_name} ({accuracy}%)"
 
    # Top & bottom 3 skill names
    top_3_names = [skill["name"] for skill in sorted_skills[:3]]
    bottom_3_skills = sorted_skills[-3:]  # Full skill objects for bottom 3
    bottom_3_names = [skill["name"] for skill in bottom_3_skills]
 
    # Format top and bottom 3 with accuracy
    top_3 = [format_with_accuracy(name) for name in top_3_names]
    bottom_3 = [format_with_accuracy(name) for name in bottom_3_names]
 
    # Extract bottom 3 resources
    bottom_3_resources = [
        {
            skill.get("name"): skill.get("educational_resources", [])
        }
        for skill in bottom_3_skills
    ]
 
    return top_3, bottom_3, bottom_3_resources
 

In [ ]:
def extract_skills(report, skill_type):
    tabs = report.get("tabs", [])
    target_tab = next((tab for tab in tabs if tab.get("tab_name") == "Focus Area"), None)
 
 
    for subject in target_tab.get("subjects", []):
        if subject.get("subject") == skill_type:
            sections = subject.get("sections", [])
            for section in sections:
                if skill_type == "Math":
                    if section.get("section_title") == "Optimizing Problem-Solving: Strengthening Core SAT Math Skills for Maximum Impact":
                        return section.get("section_details", [])
                elif skill_type == "RW":
                    if section.get("section_title") == "Targeted Skill Refinement: Enhancing Efficiency & Accuracy in SAT Reading & Writing":
                        return section.get("section_details", [])
    return []

In [ ]:
def extract_rw_and_math_skills(report):
    rw_skills = []
    math_skills = []
 
    tabs = report.get("tabs", [])
    target_tab = next((tab for tab in tabs if tab.get("tab_name") == "Accuracy Assessment"), None)
 
    for subject in target_tab.get("subjects", []):
        subject_name = subject.get("subject")
        sections = subject.get("sections", [])
        skill_section = next(
            (sec for sec in sections if sec.get("section_title") == "Skill Performance (Accuracy)"),
            None
        )
 
        skills = skill_section.get("section_details", []) if skill_section else []
       
        if subject_name == "RW":
            rw_skills = skills
        elif subject_name == "Math":
            math_skills = skills
 
    return rw_skills, math_skills

In [ ]:
# Gets bottom three resources, this is a static function
 
def get_bottom_three_resources(bottom_three_skills):
    bottom_three_resources = []
    for each_skill in bottom_three_skills:
        bottom_three_resources.append({each_skill["name"]: each_skill["educational_resources"]})
   
    return bottom_three_resources

In [ ]:
def get_skills_in_range(skills_data, start, end):
    # Sort by priority (ascending)
    sorted_skills = sorted(skills_data, key=lambda x: int(x["priority"]))
 
    # Extract slice from sorted skills (convert start/end to 0-based index)
    selected_skills = sorted_skills[start - 1:end]  # inclusive
 
    # Get only the skill names
    skill_names = [skill["name"] for skill in selected_skills]
 
    return skill_names

In [ ]:
# Create the prompt
def generate_template_from_folder(folder_path: str, prompt: str) -> str:
    """Reads input data from a given folder and returns the formatted SAT prompt."""
    input_file = os.path.join(folder_path, "Input_data.json")
 
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)
 
    report_data = data.get("report", {})
 
    student = report_data.get("user", {})
    persona = report_data.get("persona", {})
    report = report_data.get("sat_readiness_report", {})
 
 
    todays_date = datetime.today().strftime('%Y-%m-%d')
 
    rw_skills = extract_skills(report,"RW")
    rw_skills_accuraccy, math_skills_accuraccy = extract_rw_and_math_skills(report)
 
    rw_top_3, rw_bottom_3, rw_bottom_three_resources = get_top_and_bottom_skills(rw_skills, rw_skills_accuraccy)
    foundation_skills_rw = get_skills_in_range(rw_skills, 1, 3)
    elevation_skills_rw = get_skills_in_range(rw_skills, 4, 6)
    peak_skills_rw = get_skills_in_range(rw_skills, 7, 11)

 
 
    math_skills = extract_skills(report,"Math")
    math_top_3, math_bottom_3, math_bottom_three_resources= get_top_and_bottom_skills(math_skills, math_skills_accuraccy)
    foundation_skills_math = get_skills_in_range(math_skills, 1, 5)
    elevation_skills_math = get_skills_in_range(math_skills, 6, 10)
    peak_skills_math = get_skills_in_range(math_skills, 11, 14)

 
    result = get_study_plan(report["sat_score"], student["goal_score"], todays_date, student["planned_sat_date"])
    if isinstance(result, str):
        print("Error:", result)
    else:
        foundation, elevation, peak, end_date, duration, flex_duration, tone, fifth_sentence = result
 

    foundation_start_date, foundation_end_date = foundation.split(" - ")
    elevation_start_date, elevation_end_date = elevation.split(" - ")
    peak_start_date, peak_end_date = peak.split(" - ")


    # print("rw_bottom_three_resources", rw_bottom_three_resources)
 
    template_data = {
        "student_name": student['preferred_name'],
        "planned_sat_date": student["planned_sat_date"],
        "today_date": todays_date,
        "persona_name": persona["persona_name"],
        "persona_strengths": list_to_numbered_string(persona["key_strengths"]),
        "persona_challenges": list_to_numbered_string(persona["areas_of_improvement"]),
        "growth_strategies": persona["growth_strategies"],
        "current_sat_score_total":report["sat_score"],
        "current_sat_accuracy": report["accuracy_index"],
        "current_sat_time_management": report["time_management"],
        "goal_score": student["goal_score"],
        "math_top_3": list_to_numbered_string(math_top_3),
        "math_bottom_3": list_to_numbered_string(math_bottom_3),
        "rw_top_3": list_to_numbered_string(rw_top_3),
        "rw_bottom_3": list_to_numbered_string(rw_bottom_3),
        "rw_bottom_three_resources":rw_bottom_three_resources,
        "math_bottom_three_resources": math_bottom_three_resources,
        "foundation_phase_rw_skills": list_to_numbered_string(foundation_skills_rw),
        "foundation_phase_math_skills": list_to_numbered_string(foundation_skills_math),
        "foundation_date_range": foundation,
        "elevation_date_range": elevation,
        "peak_date_range": peak,
        "foundation_skills_rw": list_to_numbered_string(foundation_skills_rw),
        "elevation_skills_rw": list_to_numbered_string(elevation_skills_rw),
        "peak_skills_rw": list_to_numbered_string(peak_skills_rw),
        "foundation_skills_math": list_to_numbered_string(foundation_skills_math),
        "elevation_skills_math": list_to_numbered_string(elevation_skills_math),
        "peak_skills_math": list_to_numbered_string(peak_skills_math),
        "foundation_start_date": foundation_start_date,
        "foundation_end_date": foundation_end_date,
        "elevation_start_date": elevation_start_date,
        "elevation_end_date": elevation_end_date,
        "peak_start_date": peak_start_date,
        "peak_end_date": peak_end_date,
    }
 
    if      (prompt == "FOUNDTION_PHASE_PROMPT"):
        return FOUNDTION_PHASE_PROMPT.format(**template_data)
    elif    (prompt == "ELEVATION_PHASE_PROMPT"):
        return ELEVATION_PHASE_PROMPT.format(**template_data)
    elif    (prompt == "PEAK_PHASE_PROMPT"):
         return PEAK_PHASE_PROMPT.format(**template_data)
    elif    (prompt == "TWO_SECTIONS"):
            return TWO_SECTIONS.format(**template_data)

    return Executive_Summary_PROMPT.format(**template_data)

In [ ]:
prompts = ["Executive_Summary_PROMPT", "FOUNDTION_PHASE_PROMPT", "ELEVATION_PHASE_PROMPT", "PEAK_PHASE_PROMPT", "TWO_SECTIONS"]

In [ ]:
# prompts = ["Foundation", "Elevation", "Peak"]

In [ ]:
def invoke_and_save_response(folder_path: str, prefix: str, llm) -> None:
    result_json = {}
    for prompt_name in prompts:
        prompt = generate_template_from_folder(folder_path, prompt_name)

        response = llm.invoke(prompt)
        content = response.content if hasattr(response, "content") else response

        # print("*******************", prompt_name, "************************")
        print(f"Response: {content}")
        result_json[prompt_name] = json.loads(content)

    # Save content to a text file
    output_file_path = folder_path+"\Detailed_Study_Plan.json"
    with open(output_file_path, "w", encoding="utf-8") as f:
        f.write(json.dumps(result_json, indent=4, ensure_ascii=False))

    # # Save the response to a file with prefix
    # input_file_path = os.path.join(folder_path, "Input_data.json")
    # with open(input_file_path, "r", encoding="utf-8") as f:
    #     data = json.load(f)

    # name = data["user"]["first_name"]
    # output_filename = f"{name}_{prefix}.json"
    # output_file_path = os.path.join(folder_path, output_filename)

    # # Delete the file if it exists
    # if os.path.exists(output_file_path):
    #     os.remove(output_file_path)

    # # Directly write the content string to the file
    # with open(output_file_path, "w", encoding="utf-8") as out_f:
    #     out_f.write(content)

<>:14: SyntaxWarning: invalid escape sequence '\D'
<>:14: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Manideep S\AppData\Local\Temp\ipykernel_18172\1363857730.py:14: SyntaxWarning: invalid escape sequence '\D'
  output_file_path = folder_path+"\Detailed_Study_Plan.json"


In [ ]:
folders = ["IshanaPotti", "Meenakshi", "RohanByali"]#, "Govind", "Ishan", "Jevinn", "Ronit", "Sumedh", "Toni", "Zoha"]

In [ ]:
from tqdm import tqdm

for folder in tqdm(folders, desc="Folders Progress"):
    folder_path = r"C:\Users\Manideep S\Downloads\L@\SAT Paid Report\Users_data\{folder}".format(folder=folder)

    for i in tqdm(range(1, 2), desc=f"Processing {folder}", leave=False):
        prefix = str(i)
        invoke_and_save_response(folder_path, prefix, llm)

Folders Progress:   0%|          | 0/3 [00:00<?, ?it/s]

Response: {
  "Executive Summary": "Ishana is a curious and adaptive learner who excels at experimenting with new strategies but may face challenges in maintaining consistent focus on long-term mastery. Currently, Ishana has a solid SAT score of 1440 with an overall accuracy of 80% and a time management score of 49. This balance of strengths and areas for growth highlights the potential for significant improvement by deepening focus on key skills and developing a steady study routine. With targeted effort, Ishana is well-positioned to reach the realistic target score of 1590 by June 2025.",
  "Key Strengths": {
    "Reading & Writing": [
      "Words in Context (66%)",
      "Form, Structure, and Sense (47%)",
      "Command of Evidence - Textual (68%)"
    ],
    "Math": [
      "Nonlinear functions (78%)",
      "Nonlinear equations in one variable and systems of equations in two variables (84%)",
      "Systems of two linear equations in two variables (78%)"
    ]
  },
  "Focus Area

Folders Progress:  33%|███▎      | 1/3 [00:55<01:51, 55.59s/it]

Response: {
    "Tips to Make It Work Best": [
        "Set a clear ‘method of the week’ to focus on mastering one new strategy fully before moving on, balancing your curiosity with depth.",
        "Use short, timed practice sessions alternating between speed and accuracy to sharpen decision-making under pressure while maintaining precision.",
        "After each practice, spend 10 minutes reflecting on errors and successes to identify patterns and adjust your approach thoughtfully.",
        "Incorporate weekly self-assessments with specific goals to maintain consistent progress and prevent drifting between strategies.",
        "Track your study habits daily with a habit tracker to build discipline and reward yourself for following through, reinforcing steady routines."
    ],
    "Words of Encouragement": "Your natural curiosity and ability to learn through experimentation are powerful assets on this journey. Embracing mistakes as opportunities to grow will continue to deepen your 

Response: {
  "Executive Summary": "Meenakshi, known as Persistent Hiker, is a steady yet comfort-seeking learner who balances well-being with academic goals. Currently scoring 1510 on the SAT with 84% accuracy and a time management score of 47, she demonstrates strong consistency and calmness under pressure. Her approach favors long-term growth and emotional stability, though she may shy away from challenges that push her limits. This balance positions her well for steady improvement toward her realistic target score of 1590 by June 2025.",
  "Key Strengths": {
    "Reading & Writing": [
      "Words in Context (65%)",
      "Transitions (72%)",
      "Form, Structure, and Sense (69%)"
    ],
    "Math": [
      "Nonlinear functions (93%)",
      "Systems of two linear equations in two variables (83%)",
      "Linear equations in two variables (83%)"
    ]
  },
  "Focus Areas for Growth": {
    "Reading & Writing": [
      "Rhetorical Synthesis (94%)",
      "Command of Evidence - Qua

Folders Progress:  67%|██████▋   | 2/3 [01:56<00:59, 59.01s/it]

Response: {
    "Tips to Make It Work Best": [
        "Incorporate brief, focused micro-challenges into your study sessions that slightly push your limits, such as timed practice on nonlinear functions or words in context, to build resilience without overwhelming your balance.",
        "Use a gamified approach by setting clear, incremental goals for each study topic and reward yourself with small acknowledgments for each milestone, reinforcing steady progress and maintaining motivation.",
        "Alternate between accuracy-focused drills and timed exercises to develop adaptability, ensuring you remain deliberate but also comfortable with making decisions under time constraints.",
        "After each practice session, reflect calmly on errors by journaling what was challenging and how you approached it, framing mistakes as growth opportunities rather than stress triggers.",
        "Engage occasionally with peers who have a slightly more intense prep style in structured, time-boxed g

Response: {
  "Executive Summary": "Rohan is a tenacious yet occasionally discouraged learner, demonstrating steady effort and resilience despite challenges in seeing immediate progress. Currently, his SAT total score is 640, with an overall accuracy of 48% and a time management score of 35. His strengths lie in persistence and intrinsic motivation, which fuel consistent work even when results are slow. However, he may struggle with motivation dips when progress feels unseen and can benefit from strategic adjustments to improve efficiency. With a realistic target score of 1590 by June 2025, focused practice and regular feedback will be key to unlocking his potential and maintaining momentum on this long journey.",
  "Key Strengths": {
    "Reading & Writing": [
      "Form, Structure, and Sense (20%)",
      "Words in Context (40%)",
      "Boundaries (39%)"
    ],
    "Math": [
      "Nonlinear functions (71%)",
      "Systems of two linear equations in two variables (33%)",
      "Li

Folders Progress: 100%|██████████| 3/3 [03:03<00:00, 61.23s/it]

Response: {
    "Tips to Make It Work Best": [
        "After each practice session, create a simple visual chart tracking your accuracy and time per section to clearly see progress and identify patterns that need adjustment.",
        "Incorporate weekly reflection prompts asking what strategies worked and which didn’t, then deliberately tweak one approach to test its impact before the next session.",
        "Alternate between timed drills focused on speed and untimed practice emphasizing accuracy to build flexibility in pacing and decision-making under pressure.",
        "Log your confidence level immediately after each practice test or section to monitor emotional shifts and address dips proactively with targeted mindset strategies.",
        "Engage in peer-based, time-boxed simulations where you can compare approaches and receive feedback, helping you break habitual methods and gain fresh perspectives."
    ],
    "Words of Encouragement": "Your steady commitment and resilience 